In [ ]:
#Assuming directory structure 
# root
# |___yolov5/*
# |___MicronutrientDeficiencyData-1/*
# |___best.pt
# |___images/
#       |___testimg.jpg

# Imports

In [ ]:
%pip install -qr yolov5/requirements.txt comet_ml

Model Import

In [ ]:
import torch
model_path = './best.pt'
model = torch.hub.load('./yolov5', 'custom', path=model_path, source='local')
model.eval()
model.cpu()

target_layers = [model.model.model.model[-2]]

In [ ]:
import torch
import utils
display = utils.notebook_init()  # checks
import os

In [ ]:
import os
import numpy as np
import pandas as pd
import ast
import torch
import PIL
from tqdm.auto import tqdm
import shutil as sh
from pathlib import Path
import random
from IPython.display import Image, clear_output
import matplotlib.pyplot as plt
%matplotlib inline

EigenCAM import

In [ ]:
!pip install git+https://github.com/jacobgil/pytorch-grad-cam.git
!pip install grad-cam==1.4.6

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import torch
import cv2
import numpy as np
import requests
import torchvision.transforms as transforms
from pytorch_grad_cam import EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image
from PIL import Image

In [ ]:
COLORS = np.random.uniform(0, 255, size=(80, 3))

def parse_detections(results):
    detections = results.pandas().xyxy[0]
    detections = detections.to_dict()
    boxes, colors, names = [], [], []

    for i in range(len(detections["xmin"])):
        confidence = detections["confidence"][i]
        if confidence < 0.2:
            continue
        xmin = int(detections["xmin"][i])
        ymin = int(detections["ymin"][i])
        xmax = int(detections["xmax"][i])
        ymax = int(detections["ymax"][i])
        name = detections["name"][i]
        category = int(detections["class"][i])
        color = COLORS[category]

        boxes.append((xmin, ymin, xmax, ymax))
        colors.append(color)
        names.append(name)
    return boxes, colors, names

def draw_detections(boxes, colors, names, img):
    for box, color, name in zip(boxes, colors, names):
        xmin, ymin, xmax, ymax = box
        cv2.rectangle(
            img,
            (xmin, ymin),
            (xmax, ymax),
            color,
            2)

        cv2.putText(img, name, (xmin, ymin - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2,
                    lineType=cv2.LINE_AA)
    return img

Image Import

In [ ]:
image_url = 'images/testimg.jpg'
img = np.array(Image.open(image_url))

In [ ]:
img = cv2.resize(img, (640, 640))
rgb_img = img.copy()
img = np.float32(img) / 255
transform = transforms.ToTensor()
tensor = transform(img).unsqueeze(0)
tensor.requires_grad = True

EigenCAM image

In [ ]:
cam = EigenCAM(model, target_layers)
grayscale_cam = cam(tensor)[0, :, :]
grayscale_cam = np.maximum(grayscale_cam, 0)
grayscale_cam = grayscale_cam / grayscale_cam.max()
cam_image = show_cam_on_image(rgb_img / 255.0, grayscale_cam, use_rgb=True)

Image.fromarray(cam_image)

Saving EigenCAM image

In [ ]:
cv2.imwrite('EigenCAM-image.jpg',cv2.cvtColor(cam_image, cv2.COLOR_BGR2RGB))